In [43]:
import django, os, sys, time, resource, re, gc, shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from django.db.models import Count, Avg, F, Sum
import math

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *
from utils.utils import ars

qid = 1558
q = Query.objects.get(pk=qid)

from utils.utils import *

In [70]:

p = Project.objects.get(title="NETs")
pdocs = Doc.objects.filter(query__project=p)

techs = Technology.objects.filter(project=p).exclude(name__in=["Bioenergy","Geological Storage","Blue Carbon"])

techs = Technology.objects.filter(project=p).exclude(name__in=["Blue Carbon"])

tdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('id',flat=True)))

qtdocs = set(list(Doc.objects.filter(
    query__technology__in=techs,
    query__type__in=["default","MN"]
).values_list('id',flat=True)))

## These are all docs in the project

pdocs =  Doc.objects.filter(
    id__in=tdocs.union(qtdocs)
)

print(pdocs.distinct().count())#techs = Technology.objects.filter(project=p)


6596


In [87]:
queries = Query.objects.filter(
    technology__in=techs,type="default"
).distinct()

qts = queries.annotate(
    ndocs = Count('doc')
)

qtdf = pd.DataFrame.from_dict(list(qts.order_by('id').values(
    'id','technology__name','ndocs','title','text','database'
)))


qtdf.head(450)

,database,id,ndocs,technology__name,text,title
0,WoS,243,198,DACCS,(TS = (((capture OR extraction OR absorbtion) ...,NETs_scoping_DAC
1,WoS,286,31,BECCS,TI = ((potential* OR supply) AND (bioenergy* O...,test
2,WoS,330,660,Soil Carbon Sequestration,TS = ((soil NEAR/3 (carbon OR CO2) NEAR/3 (seq...,soil_carbon_sequestration
3,scopus,332,637,Soil Carbon Sequestration,TITLE-ABS-KEY ( ( soil W/3 ( carbon OR co2...,soil_carbon_sequestration_scopus
4,WoS,333,966,Afforestation/reforestation,TS = ((afforestation OR reforestation) AND ((c...,forests_WoS
5,scopus,334,1271,Afforestation/reforestation,TITLE-ABS-KEY ( ( afforestation OR reforesta...,forests_scopus
6,WoS,335,591,Biochar,TS = (biochar* AND ((carbon OR CO2) NEAR/3 (se...,biochar_WoS
7,scopus,336,784,Biochar,TITLE-ABS-KEY ( biochar* AND ( ( carbon OR ...,biochar_scopus
8,WoS,337,115,Ocean fertilisation,TS = (ocean NEAR/5 iron NEAR/5 (fertili*ation ...,ocean_fertilisation_WoS
9,scopus,338,122,Ocean fertilisation,TITLE-ABS-KEY ( ocean W/5 iron W/5 ( ferti...,ocean_fertilisation_scopus


In [85]:
queries.count()

223

In [3]:
trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))


allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

2460


In [71]:
## For no storage and stuff

narrowtechs = Technology.objects.filter(project=p).exclude(name__in=[
    "Bioenergy","Geological Storage","Blue Carbon","Ethics & Morals"
])

techs = Technology.objects.filter(project=p).exclude(
    name__in=["Bioenergy","Geological Storage","Blue Carbon","Ethics & Morals"]
)

ntrdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=narrowtechs
).values_list('id',flat=True)))

nttdocs = set(list(Doc.objects.filter(
    technology__in=narrowtechs
).values_list('id',flat=True)))

nttdocs = set(list(Doc.objects.filter(
    technology__in=techs,    
).exclude(query__qtype='MN').values_list('pk',flat=True)))


allreldocs =  Doc.objects.filter(
    id__in=ntrdocs.union(nttdocs)
)

print(allreldocs.count())

2092


In [12]:
spapers = pd.read_csv('../tables/unique_spreadsheet_papers.csv')

print(len(spapers))

927


In [19]:
spapers.loc[440:445,]

,Unnamed: 0,X1,X2,X3,technology
440,533,"Xie, T; Reddy, KR; Wang, C; Yargicoglu, E; Spo...",2015,Characteristics and Applications of Biochar fo...,Biochar
441,535,"Andrenelli, M.C., A. Maienza, L. Genesio, F. M...",2016,Field application of pelletized biochar: Short...,Biochar
442,536,"Mašek, Ondrej, Peter Brownsort, Andrew Cross, ...",unpublished,INFLUENCE OF BIOCHAR PRODUCTION CONDITIONS ON ...,Biochar
443,537,"Hori, Masao",2015,Carbon-negative energy system,Biochar
444,538,"Lehmann, Johannes, Jan Skjemstad, Saran Sohi, ...",2008,Australian Climate–carbon Cycle Feedback Reduc...,Biochar
445,539,"Castracani, C., A. Maienza, D.A. Grasso, L. Ge...",2015,Biochar–macrofauna Interplay: Searching for Ne...,Biochar


In [25]:
global unfound
unfound = 0

global multmatches
multmatches = 0

global found
found = 0

global indb
indb = 0

spapers = pd.read_csv('../tables/unique_spreadsheet_papers.csv')


def find_paper(x):
    
    try:
    
        tis = x['X3'].split()
        
    except:
        print(x['X3'])
        tis = str(x['X3']).split()
        
        
    try:
        py=int(x['X2'])
    except:
        py =0
    
    if len(tis) > 8:
        ti = ' '.join(tis[:8])
    else:
        ti = x['X3']
        
    docs = pdocs.filter(title__icontains=ti)
    
    global unfound
    global found
    global multmatches
    
    if docs.count() == 1:
    
        doc = docs.first()
        found +=1
    elif docs.count() > 1:
        doc = "None"
        multmatches+=1
    else:       
        di = x['X3'].replace('http://dx.doi.org/','')
        docs = pdocs.filter(wosarticle__di__iexact=di)
        if docs.count()==1:
            doc = docs.first()
            found+=1
        elif docs.count() > 1:
            doc = "None"
            multmatches+=1
        else:
            au = x['X1'].split()[0].split(',')[0]
            docs = pdocs.filter(
                docauthinst__AU__icontains=au,
                docauthinst__position=1,
                PY=py
            ).distinct()
            if docs.count()==1:
                doc = docs.first()
                found+=1
            elif docs.count() > 1:
                doc = "None"
                multmatches+=1
            else:                
                docs = pdocs.filter(
                    docauthinst__AU__icontains=au,
                    docauthinst__position__in=[1,0],
                    PY=py
                ).distinct()
                if docs.count()==1:
                    doc = docs.first()
                    found+=1
                else:
                    ti = re.findall('[A-Z]{1}\.{1}[^;]([^\.]{5,})',x['X3'])
                    if len(ti) > 0:
                        ti = ti[0]
                    else:
                        ti = "zwasdfölkjag"
                    docs = pdocs.filter(title__icontains=ti)
                    if docs.count()==1:
                        doc = docs.first()
                        found+=1
                    else:
                        doc = "None"
                        unfound+=1
                        
    try:
        doc = doc.id
    except:
        doc = doc
    
    return doc

spapers['doc_id'] = spapers.apply(lambda x: find_paper(x), axis=1)

print("no matches: {}".format(unfound))
print("multiple matches: {}".format(multmatches))
print("found {}".format(found))


spapers.head()

nan
no matches: 30
multiple matches: 28
found 869


,Unnamed: 0,X1,X2,X3,technology,doc_id
0,1,"Renforth, P (a)",2012,The potential of enhanced weathering in the UK,Enhanced weathering (terrestrial and ocean),714531
1,4,Schuiling & Krijgsman,2006,ENHANCED WEATHERING: AN EFFECTIVE AND CHEAP TO...,Enhanced weathering (terrestrial and ocean),573011
2,5,Köhler et al.,2010,Geoengineering potential of artificially enhan...,Enhanced weathering (terrestrial and ocean),156602
3,6,Taylor et al.,2016,Enhanced weathering strategies for stabilizing...,Enhanced weathering (terrestrial and ocean),630980
4,7,Hangx & Spiers,2009,Coastal spreading of olivine to control atmosp...,Enhanced weathering (terrestrial and ocean),575943


In [38]:
found_ids = [x for x in list(spapers['doc_id']) if x is not 'None']
found_docs = Doc.objects.filter(
    id__in=found_ids
)


In [34]:
newqs = Query.objects.filter(id__gt=2286,id__lt=2300,project=2,creator__username="galm")

newdocs = Doc.objects.filter(query__in=newqs)

print(newdocs.distinct().count())

270


In [56]:
gsqs = list(Query.objects.filter(
    creator__username__in=['rogers','nemet']
).values_list('id',flat=True))

doc_ids = []
rated_docs = []
unrated_docs = []
all_docs = []
p_docs = []
n_docs = []
for q in newqs:
    print('\n##################')
    print(q.title)
    qdocs = Doc.objects.filter(query=q)
    all_docs.append(list(qdocs.values_list('id',flat=True)))
    print("Documents Added: {}".format(qdocs.count()))
    # Any ratings
    ratings = DocOwnership.objects.filter(query=q,relevant__gt=0)
    print(ratings.count())
    rdocs = qdocs.filter(id__in=list(set(ratings.values_list('doc__id',flat=True))))
    rated_docs.append(list(set(list(rdocs.values_list('id',flat=True)))))
    
    # Negative ratings
    nrs = DocOwnership.objects.filter(query=q,relevant=2)
    n_docs.append(list(set(list(qdocs.filter(
        id__in=list(set(nrs.values_list('doc__id',flat=True)))
    ).values_list('id',flat=True)))))
    
    ## All relevant docs to tech
    trdocs = set(list(Doc.objects.filter(
        docownership__relevant=1,
        docownership__query__technology=q.technology
    ).exclude(query__qtype='MN').values_list('id',flat=True)))
                      #.exclude(docownership__query__id__in=gsqs)
        
    ttdocs = set(list(Doc.objects.filter(
        technology=q.technology,    
    ).exclude(query__qtype='MN').values_list('id',flat=True)))
    
    oreldocs = Doc.objects.filter(
        id__in=trdocs#.union(ttdocs)
    )
    print("Old Relevant docs: {}".format(oreldocs.count()))
    
    
    
    oreltdocs = Doc.objects.filter(
        id__in=trdocs.union(ttdocs)
    )
    print("Old Relevant docs (with tech): {}".format(oreltdocs.count()))
    
    
    

    trdocs = set(list(Doc.objects.filter(
        docownership__relevant=1,
        docownership__query__technology=q.technology
    ).exclude(docownership__query__id__in=gsqs).values_list('id',flat=True)))

    ttdocs = set(list(Doc.objects.filter(
        technology=q.technology,    
    ).exclude(query__qtype='MN').values_list('id',flat=True)))
    
    treldocs = Doc.objects.filter(
        id__in=trdocs.union(ttdocs)
    )
    print("All Relevant docs: {}".format(treldocs.count()))
    
    
    # Positive ratings
    prs = DocOwnership.objects.filter(query=q,relevant=1)
    pdocs = qdocs.filter(id__in=list(set(prs.values_list('doc__id',flat=True))))
    p_docs.append(list(set(list(pdocs.values_list('id',flat=True)))))
    
    print("New Documents Relevant: {}".format(pdocs.count()))
    
    # Costs/Potentials
    cpdocs = treldocs.filter(id__in=found_ids)
    
    print("Costs and Potentials: {}".format(cpdocs.count()))
    
    
    ncpdocs = pdocs.filter(id__in=found_ids)
    
    print("New Costs and Potentials: {}".format(ncpdocs.count()))
    
    # Unrated docs
    ur_docs = qdocs.exclude(id__in=list(set(ratings.values_list('doc__id',flat=True))))
    unrated_docs.append(list(set(list(ur_docs.values_list('id',flat=True)))))
    
    # All docs in the query that do not have a negative rating
    nrdocs = qdocs.exclude(id__in=list(set(nrs.values_list('doc__id',flat=True))))
    nids = set(list(nrdocs.values_list('id',flat=True)))
    doc_ids.append(list(nids))

print("Total Docs: {}".format(len(list(set(flatten(all_docs))))))
print("Unrated_docs: {}".format(len(list(set(flatten(unrated_docs))))))
print("Potentially relevant (unrated or relevant) Docs: {}".format(len(list(set(flatten(doc_ids))))))
print("Documents rated: {}".format(len(list(set(flatten(rated_docs))))))
print("Documents rated relevant: {}".format(len(list(set(flatten(p_docs))))))
print("Documents rated irrelevant: {}".format(len(list(set(flatten(n_docs))))))


##################
New documents uploaded by users, with user tech tags: Enhanced Weathering
Documents Added: 8
8
Old Relevant docs: 32
Old Relevant docs (with tech): 45
All Relevant docs: 19
New Documents Relevant: 2
Costs and Potentials: 3
New Costs and Potentials: 1

##################
New documents uploaded by users, with user tech tags: Blue Carbon
Documents Added: 4
0
Old Relevant docs: 2
Old Relevant docs (with tech): 18
All Relevant docs: 16
New Documents Relevant: 0
Costs and Potentials: 0
New Costs and Potentials: 0

##################
New documents uploaded by users, with user tech tags: Direct Air Capture
Documents Added: 50
50
Old Relevant docs: 217
Old Relevant docs (with tech): 220
All Relevant docs: 89
New Documents Relevant: 28
Costs and Potentials: 1
New Costs and Potentials: 1

##################
New documents uploaded by users, with user tech tags: Ocean fertilisation
Documents Added: 59
70
Old Relevant docs: 189
Old Relevant docs (with tech): 213
All Relevant docs

In [72]:
spapers[spapers['doc_id'] == "None"].reset_index()['X3']


notfound = spapers[spapers['doc_id'] == "None"].reset_index()

foundpapers = spapers[spapers['doc_id'] != "None"].reset_index()

notfound



notfound.to_excel('../tables/spreadsheet_not_found.xlsx')


#spapers[spapers['doc_id'] == "None"].groupby('technology').count()

# Go through docs, mark include1 if in allreldocs, mark include2 if in list generated in R


#spapers[spapers['doc_id'] == "None"].reset_index()['X3'][18]




In [ ]:
pdoc_dict = []

allrelids = list(allreldocs.values_list('id',flat=True))

for doc in pdocs:
    das = doc.docauthinst_set.order_by('AU','position').distinct('AU').values_list('id',flat=True)
    unique = doc.docauthinst_set.filter(id__in=das).order_by('position').values_list('AU',flat=True)
    qset = [str(x) for x in list(doc.query.filter(
            technology__in=techs,type="default"
        ).distinct().values_list('id',flat=True))]
    try:
        py = int(doc.PY)
    except:
        py = doc.PY
    if doc.id in foundpapers['doc_id'].tolist():
        sp = 1
    else:
        sp = 0
    if doc.id in allrelids:
        r = 1
        dtechs = techs.filter(
            query__docownership__doc=doc,query__docownership__relevant=1
        ) | techs.filter(doc=doc)
        #dtechs = doc.docownership_set
        dtechs = [str(x.name) for x in dtechs.distinct()]
    else:
        r = 0
        dtechs = []
        
        
    pdoc = {
        'Authors': "; ".join(unique),
        'Publication_Year': py,
        'Title': doc.title,
        'Queries': "; ".join(qset),
        'Relevant': r,
        'In_Spreadsheet': sp,
        'Technologies': '; '.join(dtechs)
    }
    pdoc_dict.append(pdoc)
    

In [94]:
print(pdocs.count())

print(len(pdoc_dict))



6596

In [90]:
big_df = pd.DataFrame.from_dict(pdoc_dict)

big_df.head()

,Authors,In_Spreadsheet,Publication_Year,Queries,Relevant,Technologies,Title
0,"Lentz, RD; Ippolito, J",0,2010.0,1192; 335; 568,0,,"Biochar, manure, or sawdust additions to calca..."
1,"Li, T.",0,2017.0,586; 585,0,,Enhanced silicate weathering of tropical shelf...
2,"Hankalin, V.",0,2017.0,367; 1458; 374; 896; 673; 672,1,BECCS; NETs - General,Performance and costs of CCS in the pulp and p...
3,"Garnett, T.",1,2017.0,618; 334,1,Afforestation/reforestation,Protein futures for Western Europe: potential ...
4,"Cong, WW; Ren, TS; Li, BG",0,2016.0,618; 333; 1208; 1207,1,Afforestation/reforestation,Changes in soil organic matter composition aft...


In [91]:
big_df.groupby(['In_Spreadsheet','Relevant']).count()

Authors  Publication_Year  Queries  Technologies  \
In_Spreadsheet Relevant                                                     
0              0            4432              4430     4432          4432   
               1            1306              1306     1306          1306   
1              0              72                72       72            72   
               1             786               786      786           786   

                         Title  
In_Spreadsheet Relevant         
0              0          4430  
               1          1306  
1              0            72  
               1           786

In [92]:
big_df.to_excel('../tables/all_considered_docs.xlsx')

qtdf.to_excel('../tables/all_queries.xlsx')

In [89]:
qtdf

,database,id,ndocs,technology__name,text,title
0,WoS,243,198,DACCS,(TS = (((capture OR extraction OR absorbtion) ...,NETs_scoping_DAC
1,WoS,286,31,BECCS,TI = ((potential* OR supply) AND (bioenergy* O...,test
2,WoS,330,660,Soil Carbon Sequestration,TS = ((soil NEAR/3 (carbon OR CO2) NEAR/3 (seq...,soil_carbon_sequestration
3,scopus,332,637,Soil Carbon Sequestration,TITLE-ABS-KEY ( ( soil W/3 ( carbon OR co2...,soil_carbon_sequestration_scopus
4,WoS,333,966,Afforestation/reforestation,TS = ((afforestation OR reforestation) AND ((c...,forests_WoS
5,scopus,334,1271,Afforestation/reforestation,TITLE-ABS-KEY ( ( afforestation OR reforesta...,forests_scopus
6,WoS,335,591,Biochar,TS = (biochar* AND ((carbon OR CO2) NEAR/3 (se...,biochar_WoS
7,scopus,336,784,Biochar,TITLE-ABS-KEY ( biochar* AND ( ( carbon OR ...,biochar_scopus
8,WoS,337,115,Ocean fertilisation,TS = (ocean NEAR/5 iron NEAR/5 (fertili*ation ...,ocean_fertilisation_WoS
9,scopus,338,122,Ocean fertilisation,TITLE-ABS-KEY ( ocean W/5 iron W/5 ( ferti...,ocean_fertilisation_scopus


In [11]:
for q in queries[:10]:
    ds = q.doc_set.all()
    dids = list(q.doc_set.all().values_list('UT',flat=True))
    ds = Doc.objects.filter(UT__in=dids)
    print(ds.count())
    ds = ds.annotate(
        qs = Count('query')
    )
    print(ds.filter(qs__lt=4).count())
    
    print(q.title)

68
0
enhanced_weathering_wos
101
0
enhanced_weathering_wos_2
637
0
soil_carbon_sequestration_scopus
4
0
DAC_extra
422
9
synonyms_all
660
0
soil_carbon_sequestration
396
0
beccs_scopus
391
6
synonyms_scopus
2
0
3A_ForestRDNarrowMissing
2
0
2D_OceanDemandpullMissing


In [12]:
q = queries.first()

ds = q.doc_set.all()

print(ds.count())

ds = ds.annotate(
    qs = Count('query')
)

print(ds.filter(qs__lt=2).count())

q.title

198
198


'NETs_scoping_DAC'

In [13]:


trdocs = set(list(pdocs.filter(
    docownership__relevant=1,
    docownership__query__technology__in=techs
).values_list('UT',flat=True)))

ttdocs = set(list(Doc.objects.filter(
    technology__in=techs
).values_list('UT',flat=True)))

print(len(trdocs))
print(len(ttdocs))

allreldocs =  Doc.objects.filter(
    UT__in=trdocs.union(ttdocs)
)

print(allreldocs.count())

print(allreldocs.order_by('PY').first().PY)
print(allreldocs.order_by('PY').first().title)

print(allreldocs.filter(PY__lt=1990).count())

1935
273
2134
1977
Production of synthetic methanol from air and water using controlled thermonuclear reactor power-I. technology and energy requirement
3
